In [2]:
import os
import tensorflow as tf

In [3]:
tf_record_filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./test_record.tfrecord"))
#f=open("./test_record.tfrecord",'rb')
tf_record_reader=tf.TFRecordReader()
_, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

In [4]:
tf_record_features = tf.parse_single_example(
    tf_record_serialized,
    features={
        'label': tf.FixedLenFeature([], tf.string),
        'width': tf.FixedLenFeature([], tf.string),
        'height': tf.FixedLenFeature([], tf.string),
        'image': tf.FixedLenFeature([], tf.string),
    })

In [5]:
tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)

tf_record_image = tf.reshape(tf_record_image, [64,64,1])

tf_record_label = tf.cast(tf_record_features['label'],tf.string)

print(tf_record_image)
print(tf_record_label)

sess=tf.Session()
#sess.run(tf_record_label)

Tensor("Reshape:0", shape=(64, 64, 1), dtype=uint8)
Tensor("ParseSingleExample/Squeeze_label:0", shape=(), dtype=string)
